This code was based on in the following references:

[1] "Grey Wolf Optimizer" published in 2013 by Seyedali Mirjalili, Seyed Mohammad Mirjalili e Andrew Lewis

[2] "A Novel Search Algorithm based on Fish School Behavior" published in 2008 by Bastos Filho, Lima Neto, Lins, D. O. Nascimento and P. Lima

[2] "An Enhanced Fish School Search Algorithm" published in 2013 by Bastos Filho and  D. O. Nascimento

[3] "Defining a Standard for Particle Swarm Optimization" published in 2007 by Bratton and Kennedy

### Imports

In [1]:
import os
import sys
import csv
import random
import copy
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
from google.colab import drive, files
%matplotlib inline
plt.rcParams["figure.figsize"] = (13,8)

### Connection with Google Drive

In [2]:
drive.mount('/content/drive', force_remount=True) #  Connect to drive
FOLDERNAME = 'POLI/decimo/tcc/Algoritms/Algoritms Notebook/' #  Set folder that have the datasts
assert FOLDERNAME is not None, "[!] Enter the foldername."
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME)) #  Locale the foldeer

%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/POLI/decimo/tcc/Algoritms/Algoritms Notebook


### Class of Wolfs and Wolf Pack with each method to execute optimization

In [3]:
from ObjectiveFunction import *
from SearchSpaceInitializer import UniformSSInitializer, OneQuarterDimWiseSSInitializer
%cd ../

class Wolf(object):
  def __init__(self, dim):
    self.pos = np.zeros(dim)
    self.score = np.inf
    self.aromatic_intensity = 0
    self.delta_pos = np.zeros(dim)
    self.delta_score = 0
    self.best_score = 0
    self.best_pos = np.zeros(dim)
    self.last_pos = np.zeros(dim)

  def update_bests(self, score=None, pos=None):
    score = self.score if score is None else score
    pos = self.pos if pos is None else pos
    if score < self.best_score or self.best_score == np.inf:
      self.best_pos = pos
      self.best_score = score
  
  def is_wolf(self, w: 'Wolf'):
    check_pos = w.pos == self.pos
    if type(check_pos) is np.ndarray:
      check_pos = check_pos.sum() == len(self.pos)

    check_last_pos = w.last_pos == self.last_pos
    if type(check_last_pos) is np.ndarray:
      check_last_pos = check_last_pos.sum() == len(self.pos)
    
    check_pos = check_last_pos and check_pos
    return check_pos and w.score == self.score and w.aromatic_intensity == self.aromatic_intensity
  
  def dist_between(self, w:'Wolf'):
    return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(self.pos, w.pos)))


class Alpha(Wolf):
  def __init__(self, dim, score, pos, last_pos, ai):
    self.score = score
    self.pos = pos
    self.last_pos = last_pos
    self.aromatic_intensity = ai


class Beta(Wolf):
  def __init__(self, dim, score, pos, last_pos, ai):
    self.score = score
    self.pos = pos
    self.last_pos = last_pos
    self.aromatic_intensity = ai


class Delta(Wolf):
  def __init__(self, dim, score, pos, last_pos, ai):
    self.score = score
    self.pos = pos
    self.last_pos = last_pos
    self.aromatic_intensity = ai


class Pack(object):
  def __init__(self, objective_function, space_initializer, n_iter, pack_size, analytic_in=False):
    self.objective_function = objective_function # função de avalição de custo
    self.space_initializer = space_initializer # posições iniciais dos peixes

    self.dim = objective_function.dim
    self.minf = objective_function.minf # limite minimo da função
    self.maxf = objective_function.maxf # limite máximo da função
    self.n_iter = n_iter

    self.pack_size = pack_size  # quantidade de peixes

    self.a = 2
    self.r1 = random.random()
    self.r2 = random.random()
    self.a1 = 2 * self.a * self.r1 - self.a
    self.c1 = 2 * self.r2
    self.alpha = None
    self.beta = None
    self.delta = None
    
    self.analytic_in = analytic_in
    self.i_net = []
    
    self.optimum_score_tracking_iter = []
    self.optimum_score_tracking_eval = []
  
  def __init_score_tracking(self):
    self.optimum_score_tracking_iter = []
    self.optimum_score_tracking_eval = []

  def __init_wolf(self, pos):
    wolf = Wolf(self.dim)
    wolf.pos = pos
    wolf.score = self.objective_function.evaluate(wolf.pos)
    self.optimum_score_tracking_eval.append(self.alpha.score)
    return wolf

  def __init_pack(self):
    self.alpha = Wolf(self.dim)
    self.beta = Wolf(self.dim)
    self.delta = Wolf(self.dim)
    self.pack = []
    
    positions = self.space_initializer.sample(self.objective_function, self.pack_size)

    for idx in range(self.pack_size):
      wolf = self.__init_wolf(positions[idx])
      self.pack.append(wolf)
    self.update_hierarchy()
    self.optimum_score_tracking_iter.append(self.alpha.score)
  
  def update_a(self, curr_iter):
    self.a = 2 - curr_iter * (2 / self.n_iter)

  def update_variables(self):
    self.r1 = random.random()
    self.r2 = random.random()
    self.a1 = 2 * self.a * self.r1 - self.a
    self.c1 = 2 * self.r2

  def update_hierarchy(self):
    for wolf in self.pack:
      if wolf.score < self.alpha.score:
        self.delta = Delta(self.dim, self.beta.score, self.beta.pos, self.beta.last_pos, self.beta.aromatic_intensity)
        self.beta = Beta(self.dim, self.alpha.score, self.alpha.pos, self.alpha.last_pos, self.alpha.aromatic_intensity)
        self.alpha = Alpha(self.dim, wolf.score, wolf.pos, wolf.last_pos, wolf.aromatic_intensity)
      elif wolf.score > self.alpha.score and wolf.score < self.beta.score:
        self.delta = Delta(self.dim, self.beta.score, self.beta.pos, self.beta.last_pos, self.beta.aromatic_intensity)
        self.beta = Beta(self.dim, wolf.score, wolf.pos, wolf.last_pos, wolf.aromatic_intensity)
      elif wolf.score > self.alpha.score and wolf.score > self.beta.score and wolf.score < self.alpha.score:
        self.delta = Delta(wolf.dim, wolf.score, wolf.pos, wolf.last_pos, wolf.aromatic_intensity)

  def collective_movement(self):
    for wolf in self.pack:
      new_pos = np.zeros((self.dim,), dtype=float)
      for dim in range(self.dim):
        self.update_variables()
        d_alpha = abs(self.c1 * self.alpha.pos[dim] - wolf.pos[dim])
        x1 = self.alpha.pos[dim] - self.a1 * d_alpha
        
        self.update_variables()
        d_beta = abs(self.c1 * self.beta.pos[dim] - wolf.pos[dim])
        x2 = self.beta.pos[dim] - self.a1 * d_beta

        self.update_variables()
        d_delta = abs(self.c1 * self.delta.pos[dim] - wolf.pos[dim])
        x3 = self.delta.pos[dim] - self.a1 * d_delta

        new_pos[dim] = (x1 + x2 + x3) / 3
        if new_pos[dim] < self.minf:
          new_pos[dim] = self.minf
        elif new_pos[dim] > self.maxf:
          new_pos[dim] = self.maxf
      score = self.objective_function.evaluate(new_pos)
      self.optimum_score_tracking_eval.append(self.alpha.score)
      wolf.last_pos = copy.copy(wolf.pos)
      wolf.pos = new_pos
      wolf.score = score
  
  def get_analytic_in(self):
    interaction_in = []
    for w_i in self.pack:
      interaction_w = []
      is_leader = self.alpha.is_wolf(w_i) or self.beta.is_wolf(w_i) or self.delta.is_wolf(w_i)
      for w_j in self.pack:
        euclidean_distance = w_i.dist_between(w_j) if is_leader else 0
        i_net = euclidean_distance
        interaction_w.append(i_net)
      interaction_in.append(interaction_w)
    self.i_net.append(interaction_in)

  def optimize(self):
    self.__init_score_tracking()
    self.__init_pack()

    for i in range(self.n_iter):
      self.update_a(i)
      self.collective_movement()
      self.update_hierarchy()
      self.optimum_score_tracking_iter.append(self.alpha.score)
      if self.analytic_in:
        self.get_analytic_in()

/content/drive/MyDrive/POLI/decimo/tcc/Algoritms


### Validate GWO

Validate learning in each iteration

In [4]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for d in [15, 30]:
    print (f"starting VGWO ({d})")
    search_space_initializer = UniformSSInitializer()
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + f"{d}d" + os.sep
    num_exec = 1
    pack_size = 30
    num_iterations = 1000

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, PermFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction, Levi13Function]
    regular_functions = unimodal_funcs + multimodal_funcs

    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + "/GWO_iter.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['opt', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_iterations)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(d)
      start = time.time()
      bests_iter, bests_eval = run_experiments(num_iterations, pack_size, func, search_space_initializer)
      end = time.time()
      row_csv = ['GWO', func.function_name, (end - start)] + [b for b in bests_iter[:num_iterations]]
      writer_csv.writerow(row_csv)
      print(func.function_name, end - start, bests_iter[-1])
      
    f_handle_csv.close()

def run_experiments(n_iter, pack_size, objective_function, search_space_initializer):
  opt1 = Pack(objective_function=objective_function, space_initializer=search_space_initializer,
              n_iter=n_iter, pack_size=pack_size)
  opt1.optimize()
  return opt1.optimum_score_tracking_iter, opt1.optimum_score_tracking_eval

main()

starting VGWO (15)
Sphere 5.617436647415161 3.840698386966459e-100
RotatedHyperEllipsoid 6.9594221115112305 1.2949848218727163e-106
Rosenbrock 4.41228461265564 12.152998042440283
Dixon-Price 4.592839241027832 0.010364521829726914
Perm 12.634944677352905 0.001212394817342223
Quartic-Noise 5.038585186004639 0.00044017816814866367
Generalized-Shwefel 4.481478214263916 3711.8102439233803
Rastrigin 4.156270503997803 2.842170943040401e-14
Ackley 4.808553218841553 7.549516567451064e-15
Griewank 3.940181255340576 0.0
Levi 5.805164098739624 0.013506670560559509
Levi-13 6.266216516494751 0.2984499433583869
starting VGWO (30)
Sphere 6.562513589859009 9.208143694324608e-65
RotatedHyperEllipsoid 12.3142671585083 9.731686861363932e-66
Rosenbrock 6.806604623794556 26.17612667689869
Dixon-Price 7.31370997428894 0.0023237283826464893
Perm 46.77654790878296 9.875833641726027
Quartic-Noise 7.620934963226318 0.001092190908009158
Generalized-Shwefel 6.51049542427063 5856.0835858300325
Rastrigin 6.920244693

Validate bests results in multiple sessions of iterations

In [5]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for d in [15, 30]:
    print (f"starting VGWO ({d})")
    search_space_initializer = UniformSSInitializer()
    file_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Executions" + os.sep + f"{d}d" + os.sep
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + f"{d}d" + os.sep
    num_exec = 30
    pack_size = 30
    num_iterations = 1000

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, PermFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction, Levi13Function]
    regular_functions = unimodal_funcs + multimodal_funcs

    # regular_functions = [RastriginFunction, RosenbrockFunction]

    # Notice that for CEC Functions only the following dimensions are available:
    # 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100
    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + "/GWO_exec.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['opt', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_exec)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(d)
      
      start = time.time()
      runs = run_experiments(num_iterations, pack_size, num_exec, func,
                            search_space_initializer, file_path)
      end = time.time()
      row_csv = ['GWO', func.function_name, (end - start)] + [r for r in runs]
      writer_csv.writerow(row_csv)
      
    f_handle_csv.close()


def run_experiments(n_iter, pack_size, num_runs, objective_function,
                    search_space_initializer, save_dir):
  alg_name = "GWO"
  console_out = "Algorithm: {} Function: {} Execution: {} Best Cost: {}"
  if save_dir:
    create_dir(save_dir)
    f_handle_cost_iter = open(save_dir + "/GWO_" + objective_function.function_name + "_cost_iter.txt", 'w+')
    f_handle_cost_eval = open(save_dir + "/GWO_" + objective_function.function_name + "_cost_eval.txt", 'w+')
    runs = []

  for run in range(num_runs):
    opt1 = Pack(objective_function=objective_function, space_initializer=search_space_initializer,
                n_iter=n_iter, pack_size=pack_size)
    
    opt1.optimize()
    runs.append(opt1.alpha.score)
    print (console_out.format(alg_name, objective_function.function_name, run+1, opt1.alpha.score))

    temp_optimum_cost_tracking_iter = np.asmatrix(opt1.optimum_score_tracking_iter)
    temp_optimum_cost_tracking_eval = np.asmatrix(opt1.optimum_score_tracking_eval)

    if save_dir:
      np.savetxt(f_handle_cost_iter, temp_optimum_cost_tracking_iter, fmt='%.4e')
      np.savetxt(f_handle_cost_eval, temp_optimum_cost_tracking_eval, fmt='%.4e')

  if save_dir:
    f_handle_cost_iter.close()
    f_handle_cost_eval.close()

  return runs

main()

starting VGWO (15)
Algorithm: GWO Function: Sphere Execution: 1 Best Cost: 3.112592727369915e-100
Algorithm: GWO Function: Sphere Execution: 2 Best Cost: 5.395613806839329e-99
Algorithm: GWO Function: Sphere Execution: 3 Best Cost: 2.9712786419537425e-99
Algorithm: GWO Function: Sphere Execution: 4 Best Cost: 9.938825899718963e-102
Algorithm: GWO Function: Sphere Execution: 5 Best Cost: 6.957934275230295e-100
Algorithm: GWO Function: Sphere Execution: 6 Best Cost: 4.383018037927669e-98
Algorithm: GWO Function: Sphere Execution: 7 Best Cost: 5.159385479162713e-100
Algorithm: GWO Function: Sphere Execution: 8 Best Cost: 2.4356974504898608e-98
Algorithm: GWO Function: Sphere Execution: 9 Best Cost: 6.464459136166563e-101
Algorithm: GWO Function: Sphere Execution: 10 Best Cost: 9.884317713089283e-103
Algorithm: GWO Function: Sphere Execution: 11 Best Cost: 6.888418683455961e-99
Algorithm: GWO Function: Sphere Execution: 12 Best Cost: 4.9172090076854106e-99
Algorithm: GWO Function: Sphere E